# 概要
* 複数インデックスのマージにいよいよトライする
* treeは自動で要約つけてくれるの？？めっちゃ便利やん
* 訳としては完璧やん...
* num_outputは2048も対応している→ダイでエラー発生
* 親の要約が変なのが惜しい
* b_indexがおかしい
* #prompt_helperで重複を許すようにすれば自然な日本語になるはず#！！
* そもそもこういう問題があるからチュートリアルはkeyword_tableを使っていたのか
* indexを作っただけではembeddingしていないことに注意！！
* 親がkeyword_tableで子をtreeにするべきなのでは？
* 多分これ以外は組み合わせるけど、組み合わせ的にうまく動かない
* https://gpt-index.readthedocs.io/en/latest/how_to/composability.html#querying-the-top-level-index

# 参考
* そもそもtree構造が前提なのでtreeのindexを確認する
* https://gpt-index.readthedocs.io/en/latest/how_to/composability.html
* https://note.com/npaka/n/n5d4ff4380055
* https://note.com/oshizo/n/n137aaa2c29d4
* 日本語としてparseされない...

# 結局
* 一応ダイとドラゴンボールで動いてる
* ただ精度はいまいち
* そもそも下部構造のtreeの精度がいまいち

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [3]:
from gpt_index import LLMPredictor, GPTSimpleVectorIndex, SimpleDirectoryReader

In [4]:
# OpenAIのAPIのラッパー
from langchain.llms import OpenAI

# 利用例

In [5]:
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader,
    GPTTreeIndex
)

from llama_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper
)

from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 1024+512
# set maximum chunk overlap
max_chunk_overlap = 0
# separatorが"/n/n"だとチャンクサイズの計算でエラーになるっぽい。"。"の場合は動く模様。
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="。", chunk_size_limit=512)

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_output))

# build index
documents = SimpleDirectoryReader('story/dragonball').load_data()
d_index = GPTTreeIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

documents = SimpleDirectoryReader('story/dai').load_data()
b_index = GPTTreeIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
INFO:root:> Building index from nodes: 2 chunks
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Write a summary of the following. Try to use only the information provided. Try to include as many key details as possible.\\n\\n\\n\\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\n\\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u306f\\u3042\\u308b\\u65e5\\u3001\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u6765\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3046\\n\\u305d\\u3053\\u3067\\u30017\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u3001\\u3069\\u3093\\u306a\\u9858\\u3044\\u3067\\u3082\\u4e00\\u3

DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openai.com:443
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=26112 request_id=5e841d2cbd8b75827277101de78aac4a response_code=200
DEBUG:root:
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めて神龍を呼び出すことを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。その道中で武術の達人・亀仙人の下で8か月間の修行を積み、天下一武道会に出場し準優勝を果たす。3年後の天下一武道会でも準優勝を果たし、修行の旅へと出発する。

ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得て潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。その後、悟空は神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。

サイヤ人編では、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空は孫悟飯を助けるため、ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空はドラゴンボールによ

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=32066 request_id=a554576721ccb4fdf6b3ddc93dd0df06 response_code=200
DEBUG:root:
悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落としていったが、悟空は伝説の戦士・超サイヤ人へと覚醒し、フリーザを倒す。ナメック星での闘いから約1年後、謎の超サイヤ人によって撃退される。トランクスから渡された特効薬によって悟空は生還し、来るべき日に備えて3年間各々に修行してその日を迎える。その日に現れたのは、セルという究極の人造人間であり、悟空らの想定を遥かに超えた戦士が続々と現れた。セルは地球の命運を賭けた武道会「セルゲーム」の開催を全世界に宣言する。悟空はこのセルとの闘いの中で地球を守るために命を落とすが、悟飯が超サイヤ人2へと覚醒し、父・悟空の幻影と共にかめはめ波を放ちセルを撃破する。セルゲームを制した悟飯たちは、ドラゴンボールによりセルに殺された人々を蘇生させる。セルゲームより約7年後、悟空は天下一武道会に出場するために占いババの力によって1日だけこの世に戻る。天下一武道会の最中、悟空たちは界王神から、恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。
DEBUG:openai:message='Request to OpenAI API' method=post path=htt

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=32978 request_id=099f4aa3dcbb4efd2fe59398cce18ee7 response_code=200
DEBUG:root:
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、友達のモンスターたちと共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。アバンの遺志を継いだダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立っていく。ロモス王国とパプニカ王国を訪れ、魔王軍六大軍団と戦い、勝利を収める。しかし、ダイは次第に自分自身の力が人間として超絶したものであることによる他者からの視線に苦悩するようになる。そんなダイの前に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であること、さらには自らの息子であることを告げた。バランはダイに魔王軍に加わり自分と共に人間を滅ぼすよう促すが、ダイは拒否。最後の手段としてメガンテを発動したポップの犠牲がダイの記憶を蘇らせ、バランは何処かに姿を消す。
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=26311 request_id=5e73da1f91d1430530141915edc18bd8 response_code=200
DEBUG:root:
レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち入ることが出来ずにいた。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、奇跡を起こす神々のアイテム「神の涙」であったゴメちゃんの最期の力によって、世界中の人々が協力し、爆発は阻止される。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。制御を失い崩壊しながら浮上を続けるバーンパレスごと宇宙空間で繰り広げられる両者の最後の激闘は、ダイの勝利で終わる。地上に帰還したダイが仲間たちに迎えられ、キルバーンが現れ、自らの正体を明かす。キルバーンはバーンをも倒したダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。ダイとポップは核晶を空高く運んでいくが、土壇場でダイはポップを地上に戻し、一人爆発の閃光の中に消えた。数週間の捜索を経てもなお、ダイの姿は見つからなかった。
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Write a summary of the following. Try to

In [18]:
b_index.docstore.docs['c11828fb-27fe-4607-bd05-61fe7c407177'].text="""
十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。それから十数年後、デルムリン島に住む少年ダイは、勇者になることを夢見ていた。ある日、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受けて、短期間のうちに秘められた力を開花させていく。15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲するが、ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。ダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立ち、ロモス王国とパプニカ王国を占拠した魔王軍六大軍団と戦い、勝利を収める。
ダイは人間からの視線に苦悩していたが、その前に竜騎将バランが現れ、ダイが神々が作った伝説の存在「竜の騎士」であることを告げた。バランはダイに魔王軍に加わるよう促したが、ダイは拒否した。その後、ポップがメガンテを発動して犠牲になり、ダイの記憶が蘇った。レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も新たな力を蓄え、ハドラーは妖魔司教ザボエラの手で自らを超魔生物にさせる。ダイ一行は魔王軍の本拠地である鬼岩城を破壊したが、バーンパレスには魔宮の門によって閉ざされていた。バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。
ダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった。ダイは竜の紋章の力でバーンを追い詰め、バーンはミストバーンに預けていた肉体を使って真の姿となる。ダイは竜の魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。最後の激闘はダイの勝利で終わり、地上に帰還したダイが仲間たちに迎えられるが、キルバーンが現れ、自らの正体を明かす。キルバーンはバーンの監視と暗殺を本来の目的としていたのだった。キルバーンはダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。ダイとポップは核晶を空高く運んでいくが、土壇場でダイはポップを地上に戻し、一人爆発の閃光の中に消えた。数週間の捜索を経てもなお、ダイの姿は見つからなかったが、ロン・ベルクは持ち主の命と繋がっているダイの剣の、魔法宝玉が輝き続けていることから、ダイがどこかで生きていることを確信し、ポップはいつかダイが戻ってくる日まで、世界の平和を自分たちで守り抜くことを誓い、再びダイを探す旅に出た。
"""

In [25]:
d_index.docstore.docs['7d7f7c55-d3e7-4e88-b3dc-7fe51c33a19c'].text="""
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。その後、悟空は神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。サイヤ人編では、ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。
悟空は孫悟飯を助けるため、宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒す。その後、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。フリーザを倒し、地球の神と殺された仲間たちを甦らせることに成功したが、その約1年後には謎の超サイヤ人によって撃退されたフリーザとその一味が再び地球を襲撃する。トランクスと名乗るその青年は、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる。
セルゲームと呼ばれる武道会を開催することで、地球の命運を賭けたセルは、人造人間17号と人造人間18号を吸収して完全体となる。悟空らは、精神と時の部屋で修行を行い、セルとの闘いの中で地球を守るために命を落とす。その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒し、セルを撃破する。ドラゴンボールによりセルに殺された人々を蘇生させるが、悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ。

約7年後、高校生に成長した悟飯が天下一武道会に出場することを知った悟空は、自らも出場するために占いババの力によって1日だけこの世に戻る。天下一武道会の最中、悟空たちは界王神から、恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。復活した魔人ブウにより悟飯やベジータが倒され、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌し、悟天とトランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスや、潜在能力を解放し、パワーアップを遂げて帰ってきた悟飯らが応戦するが、苦戦を強いられる。悟空とベジータが界王神界で真の姿となった魔人ブウとの最終決戦に臨み、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。

10年後、悟空は孫のパンと共に天下一武道会に久しぶりに出場し、魔人ブウの生まれ変わりである少年・ウーブと出会う。悟空はウーブと共に見果てぬ強さを追い求めて修行に旅立ち、物語は幕を閉じる。

"""

# keyword_table index

In [26]:
kt_index = GPTKeywordTableIndex([b_index, d_index], prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> Adding chunk: 
十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わ...
DEBUG:root:> Adding chunk: ダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立ち、ロモス王国とパプニカ王国を占拠した魔王...
DEBUG:root:> Adding chunk: バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守...
DEBUG:root:> Adding chunk: ダイは竜の紋章の力でバーンを追い詰め、バーンはミストバーンに預けていた肉体を使って真の姿となる...
DEBUG:root:> Adding chunk: 数週間の捜索を経てもなお、ダイの姿は見つからなかったが、ロン・ベルクは持ち主の命と繋がっている...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords from the text. Avoid stopwords.---------------------\\n\\n\\u5341\\u6570\\u5e74\\u524d\\u3001\\u9b54\\u738b\\u30cf\\u30c9\\u30e9\\u30fc\\u7387\\u3044\\u308b\\u9b54\\u738b\\u8ecd\\u3068\\u52c7\\u8005\\u7387\\u3044\\u308b\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u6226\\u3044\\u306f\\u3001\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u52dd\\u5229\\u3067\\u7d42\\u308f\\u308a\\u

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2657 request_id=43d837c44397a39947fd0141f7b51c91 response_code=200
DEBUG:root:
KEYWORDS: バラン, ダイ一行, ハドラー, 黒の核晶, バーン, バーンパレス, フローラ, レオナ, 大破邪呪文, ミナカトール, キルバーン, カールのまもり, アバン
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords from the text. Avoid stopwords.---------------------\\n\\u30c0\\u30a4\\u306f\\u7adc\\u306e\\u7d0b\\u7ae0\\u306e\\u529b\\u3067\\u30d0\\u30fc\\u30f3\\u3092\\u8ffd\\u3044\\u8a70\\u3081\\u3001\\u30d0\\u30fc\\u30f3\\u306f\\u30df\\u30b9\\u30c8\\u30d0\\u30fc\\u30f3\\u306b\\u9810\\u3051\\u3066\\u3044\\u305f\\u8089\\u4f53\\u3092\\u4f7f\\u3063\\u3066\\u771f\\u306e\\u59ff\\u3068\\u306a\\u308b\\u3002\\u30c0\\u30a4\\u306f

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2959 request_id=9bdb4978d2b61a98a96e071f9c090ccd response_code=200
DEBUG:root:
KEYWORDS: 孫悟空, 少年, ブルマ, ドラゴンボール, 神龍, ウーロン, ヤムチャ, 亀仙人, 修行, 天下一武道会, ピッコロ大魔王, クリリン
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords from the text. Avoid stopwords.---------------------\\n\\u609f\\u7a7a\\u306f\\u4ec7\\u3092\\u8a0e\\u3064\\u305f\\u3081\\u3001\\u30e4\\u30b8\\u30ed\\u30d9\\u30fc\\u3084\\u4ed9\\u732b\\u30ab\\u30ea\\u30f3\\u306e\\u5354\\u529b\\u3092\\u5f97\\u3066\\u547d\\u3092\\u8ced\\u3057\\u3066\\u6f5c\\u5728\\u3059\\u308b\\u529b\\u3092\\u5f15\\u304d\\u51fa\\u3057\\u3001\\u30d4\\u30c3\\u30b3\\u30ed\\u5927\\u9b54\\u738b\\u306b\\u95d8\\u30

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=4672 request_id=beace3e59bdbc790e8f0d0aa5824f4ea response_code=200
DEBUG:root:
KEYWORDS: セルゲーム, 武道会, 人造人間, 悟空, 精神, 時の部屋, 修行, 地球, 悟飯, 超サイヤ人, ドラゴンボール, 悪人, 天下一武道会, 占いババ, 界王神, 魔人ブウ, 封印
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords from the text. Avoid stopwords.---------------------\\n\\u5fa9\\u6d3b\\u3057\\u305f\\u9b54\\u4eba\\u30d6\\u30a6\\u306b\\u3088\\u308a\\u609f\\u98ef\\u3084\\u30d9\\u30b8\\u30fc\\u30bf\\u304c\\u5012\\u3055\\u308c\\u3001\\u609f\\u7a7a\\u306f\\u3042\\u306e\\u4e16\\u306b\\u5e30\\u3063\\u305f\\u305f\\u3081\\u3001\\u5730\\u7403\\u306e\\u547d\\u904b\\u306f\\u609f\\u7a7a\\u306e\\u6b21\\u7537\\u30fb\\u5b6b\\u6

In [27]:
dir(kt_index)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_document_to_index',
 '_async_add_document_to_index',
 '_async_extract_keywords',
 '_build_fallback_text_splitter',
 '_build_index_from_documents',
 '_delete',
 '_docstore',
 '_embed_model',
 '_extract_keywords',
 '_get_nodes_from_document',
 '_include_extra_info',
 '_index_registry',
 '_index_struct',
 '_insert',
 '_is_protocol',
 '_llm_predictor',
 '_preprocess_query',
 '_process_documents',
 '_prompt_helper',
 '_text_splitter',
 '_update_index_registry_and_docstore',
 '_use_async',
 '_validate_documents',

In [28]:
kt_index.docstore.docs.keys()

dict_keys(['c11828fb-27fe-4607-bd05-61fe7c407177', '7d7f7c55-d3e7-4e88-b3dc-7fe51c33a19c', 'd6c84ea0-f4f4-4175-adfc-b4e3104c10d7'])

In [29]:
kt_index.docstore.docs['d6c84ea0-f4f4-4175-adfc-b4e3104c10d7']

KeywordTable(text=None, doc_id='d6c84ea0-f4f4-4175-adfc-b4e3104c10d7', embedding=None, extra_info=None, table={'ポップ': {5997965547592609434, 537720479949755445}, '力': {537720479949755445}, '平和': {5403684715214422297, 537720479949755445}, 'アバン': {2326401442493289513, 537720479949755445}, 'マホカトール': {537720479949755445}, 'モンスター': {537720479949755445}, '紋章': {537720479949755445}, '勇者': {537720479949755445}, 'ハドラー': {2326401442493289513, 5997965547592609434, 537720479949755445}, 'ダイ': {5403684715214422297, 8583824573504485176, 5997965547592609434, 537720479949755445}, 'ロモス王国': {5997965547592609434}, 'バーンパレス': {2326401442493289513, 5997965547592609434}, 'メガンテ': {5997965547592609434}, 'パプニカ王国': {5997965547592609434}, '魔宮': {5997965547592609434}, '神々': {5997965547592609434}, '妖魔司教ザボエラ': {5997965547592609434}, '伝説': {5997965547592609434}, '鬼岩城': {5997965547592609434}, 'ゴメちゃん': {5997965547592609434}, '竜騎将バラン': {5997965547592609434}, 'バーン': {2326401442493289513, 8583824573504485176}, 'ダイ一行': {2326

In [41]:
from llama_index.composability import ComposableGraph
graph = ComposableGraph.build_from_index(kt_index)

In [54]:
query_configs = [
    {
        "index_struct_type": "tree",
        "query_mode": "default",
        "query_kwargs": {
            "child_branch_factor": 1
        }
    },
    {
        "index_struct_type": "keyword_table",
        "query_mode": "default",
        "query_kwargs": {}
    }
]

In [55]:
graph.query("フリーザを倒したのは誰か日本語で教えて", query_configs=query_configs)
#graph.query("魔人ブウを日本語",)

INFO:root:> Starting query: フリーザを倒したのは誰か日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30d5\\u30ea\\u30fc\\u30b6\\u3092\\u5012\\u3057\\u305f\\u306e\\u306f\\u8ab0\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (2): api.openai.com:443
DEBUG:urllib3.connectionpool:https://api.openai.com:443 

DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1814 request_id=507bc217475b29a76b66d95e277b0416 response_code=200
DEBUG:root:
ANSWER: 2. This summary was selected because it mentions that Goku was able to defeat Frieza with the help of the Super Saiyan transformation.
DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.
---------------------
(1)  孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めて神龍を呼び出すことを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。その道中で武術の達人・亀仙人の下で8か月間の修行を積み、天下一武道会に出場し準優勝を果たす。3年後の天下一武道会でも準優勝を果たし、修行の旅へと出発する。  ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得て潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。その後、悟空は神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う

(2)  悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着し

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 554
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3616 request_id=0bc996c8c86cddef079c6f115d84fa7f response_code=200
DEBUG:root:
ANSWER: 8 
This summary was selected because it explains how Goku sacrificed his life to defeat Frieza in order to protect the Earth. It also mentions how Goku's son, Gohan, was able to awaken to Super Saiyan 2 and use the Kamehameha wave with the help of Goku's spirit to defeat Cell.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.
---------------------
(1) 悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する

(2)   フリーザ編 地球の神と殺された仲間たちを甦らせるため、重傷で入院中の悟空に代わり、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かう。だが、そこには地球で闘ったベジータや、界王すら畏怖する宇宙の帝王・フリーザとその一味が不老不死

Response(response='悟空がフリーザを倒した。', source_nodes=[SourceNode(source_text='悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。フリーザを倒し、地球の神と殺された仲間たちを甦らせることに成功したが、その約1年後には謎の超サイヤ人によって撃退されたフリーザとその一味が再び地球を襲撃する。トランクスと名乗るその青年は、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる', doc_id='7d7f7c55-d3e7-4e88-b3dc-7fe51c33a19c', extra_info=None, node_info={'start': 638, 'end': 991}, similarity=None), SourceNode(source_text='\n\n人造人間17号と人造人間18号を吸収することで完全体となったセルは地球の命運を賭けた武道会「セルゲーム」の開催を全世界に宣言する。悟空らは天界にある1日で1年の修行が行えるも過酷な環境に晒される「精神と時の部屋」で修行し、強さを増してセルゲームに臨むが、悟空はこのセルとの闘いの中で地球を守るために命を落とす', doc_id='f3c5559a-b1fb-4bac-a858-b4818060fa15', extra_info=None, node_info=None, similarity=None)], extra_info=None)

In [56]:
graph.query("ハドラーは最後どうなったのか日本語で教えて", query_configs=query_configs)

INFO:root:> Starting query: ハドラーは最後どうなったのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30cf\\u30c9\\u30e9\\u30fc\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='Open

DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.
---------------------
(1)  十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、友達のモンスターたちと共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける

(2)  レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち入ることが出来ずにいた。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、奇跡を起こす神々のアイテム「神の涙」であったゴメちゃんの最期の力によって、世界中の人々が協力し、爆発は阻止される。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。制御を失い崩壊しながら浮上を続けるバーンパレスごと宇宙空間で繰り広げられる両者の最後の激闘は、ダイの勝利で終わる

(3) Ron Burk is certain that Dai is still alive somewhere, due to the magical gem on Da

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 641
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3188 request_id=f901e04f88d6509152d9eaf95a4ecfc0 response_code=200
DEBUG:root:
ANSWER: 5 
This summary was selected because it provides the most detailed information about what happened to Haldor in the end. It explains how Haldor rebelled against Burn and joined forces with the group, as well as how he was used as a pawn in the battle against Burn. It also explains how Haldor was saved by Carl's Mamono, and how he was ultimately defeated by Burn.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.
---------------------
(1)   大魔王との対峙、交戦 軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も魔界の名工ロン・ベルクによる「ダイの剣」の作成や、アバンの戦友であった大魔道師マトリフと拳聖ブロキーナへの師事といった出来事を経て、新たな力を

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 417
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1578 request_id=fa578617d3b8b213948a664596a61426 response_code=200
DEBUG:root:
ANSWER: 2 
This summary was selected because it explains how Dai defeats the revived Hadoraa in a final battle, which is the answer to the question.
DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.
---------------------
(1)  十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、友達のモンスターたちと共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける

(2)  レオナは世界各国の王に協力を呼

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 641
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3332 request_id=7135a87f27e40ba62f4496e1ef42b5ba response_code=200
DEBUG:root:
ANSWER: 5 
This summary was selected because it provides the most detailed information about what happened to Haldor in the end. It explains how Haldor rebelled against Burn and joined forces with the group, as well as how he was used as a pawn in the battle against Burn. It also explains how Haldor was saved by Carl's Mamono, and how he was ultimately defeated by Burn.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.
---------------------
(1)   大魔王との対峙、交戦 軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も魔界の名工ロン・ベルクによる「ダイの剣」の作成や、アバンの戦友であった大魔道師マトリフと拳聖ブロキーナへの師事といった出来事を経て、新たな力を

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 399
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1300 request_id=53f53bc009268894303e0ae43e6970fc response_code=200
DEBUG:root:
ANSWER: 2 
This summary was selected because it explains how Dai defeats the revived Hadoraa, which is the answer to the question.
DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.
---------------------
(1)  十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、友達のモンスターたちと共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける

(2)  レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 641
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2641 request_id=e4f65913da9ab7784f7714e05af3b171 response_code=200
DEBUG:root:
ANSWER: 5 
This summary was selected because it provides the most detailed information about what happened to Haldor in the end. It explains how Haldor rebelled against Burn and joined forces with the group, as well as how he was used as a pawn in the battle against Burn. It also explains how Haldor was saved by Carl's Mamono, and how he was ultimately defeated by Burn.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.
---------------------
(1)   大魔王との対峙、交戦 軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も魔界の名工ロン・ベルクによる「ダイの剣」の作成や、アバンの戦友であった大魔道師マトリフと拳聖ブロキーナへの師事といった出来事を経て、新たな力を

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=5366 request_id=8735db01b04bed1b15205a7caf603904 response_code=200
DEBUG:root:

ハドラーは、アバンの使徒との決着を願うために激闘を繰り広げた末に、バーンパレスを倒し、最終的に勝利し、その結果、アバンの使徒を撃退し、世界を救うことに成功したということになります。
DEBUG:root:> Refined response: 

ハドラーは、アバンの使徒との決着を願うために激闘を繰り広げた末に、バーンパレスを倒し、最終的に勝利し、その結果、アバンの使徒を撃退し、世界を救うことに成功したということになります。
INFO:root:> [query] Total LLM token usage: 12902 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens


Response(response='\n\nハドラーは、アバンの使徒との決着を願うために激闘を繰り広げた末に、バーンパレスを倒し、最終的に勝利し、その結果、アバンの使徒を撃退し、世界を救うことに成功したということになります。', source_nodes=[SourceNode(source_text='バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。\nダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった', doc_id='c11828fb-27fe-4607-bd05-61fe7c407177', extra_info=None, node_info={'start': 659, 'end': 1013}, similarity=None), SourceNode(source_text='\n\n最終決戦\n邪魔者を退けたバーンはバーンパレスを飛行させ、世界各地への爆撃を実行する。一方、カール王国女王のフローラの協力も得て辛うじて生き延びたダイ一行は、5人目のアバンの使徒になったレオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にアバンの使徒との決着を願うハドラーと親衛騎団も乱入し、激闘を繰り広げる', doc_id='e10be0e1-ed27-4bf2-9e51-c19d9f9f90ef', extra_info=None, node_info=None, similarity=None), SourceNode(source_text='ダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立ち、ロモス王国とパプニカ王国を占拠

In [57]:
graph.query("バランは最後どうなったのか日本語で教えて", query_configs=query_configs)

INFO:root:> Starting query: バランは最後どうなったのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30d0\\u30e9\\u30f3\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API r

INFO:root:>[Level 0] Selected node: [2]/[2]
DEBUG:root:>[Level 0] Node [2] Summary text:  レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.\\n---------------------\\n(1)   \\u5927\\u9b54\\u738b\\u3068\\u306e\\u5bfe\\u5cd9\\u3001\\u4ea4\\u6226 \\u8ecd\\u56e3\\u95774\\u4eba\\u3092\\u6b20\\u304d\\u3001\\u9b54\\u738b\\u8ecd\\u306e\\u6226\\u529b\\u306f\\u5927\\u304d\\u304f\\u4f4e\\u4e0b\\u3057\\u305f\\u3053\\u3068\\u304b\\u3089\\u3001\\u30ec\\u30aa\\u30ca\\u306f\\u4e16\\u754c\\u5404\\u56fd\\u306e\\u738b\\u306b\\u5354\\u529b\\u3092\\u547c\\u3073\\u304b\\u3051\\u3001\\u9b54\\u738b\\u8ecd\\u3078\\u306e\\u6c7a\\u6226\\u306e\\u6e96\\u5099\\u3092\\u9032\\u3081\\u308b\\u3002\\

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 642
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=4573 request_id=f39b8b0c6ec9fbdf16ef8af641824fe8 response_code=200
DEBUG:root:
ANSWER: 6
This summary was selected because it provides the most information about what happened to Balan in the end. It explains how Balan sacrificed himself to protect the others, and how his son, who had been secretly training, returned to help them defeat the enemy. It also explains how Balan's power was used to confront the enemy, and how he was ultimately defeated.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.
---------------------
(1)   大魔王との対峙、交戦 軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も魔界の名工ロン・ベルクによる「ダイの剣」の作成や、アバンの戦友であった大魔道師マトリフと拳聖ブロキーナへの師事といった出来事を経て、新たな力

Response(response='バランは、アバンがハドラーを倒した後、キルバーンの罠を解除したことで、ダイと共に無事生還したことになります。', source_nodes=[SourceNode(source_text='バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。\nダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった', doc_id='c11828fb-27fe-4607-bd05-61fe7c407177', extra_info=None, node_info={'start': 659, 'end': 1013}, similarity=None), SourceNode(source_text='ダイはハドラーを倒すが、その直後にキルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった。\n\n父・バランから受け継いだ竜の紋章の力でダイはバーンを追い詰めるが、バーンはミストバーンに預けていた肉体を使って真の姿となる', doc_id='e10be0e1-ed27-4bf2-9e51-c19d9f9f90ef', extra_info=None, node_info=None, similarity=None)], extra_info=None)

In [58]:
query_configs2 = [
    {
        "index_struct_type": "tree",
        "query_mode": "default",
        "query_kwargs": {
            "child_branch_factor": 2
        }
    },
    {
        "index_struct_type": "keyword_table",
        "query_mode": "default",
        "query_kwargs": {}
    }
]

In [59]:
graph.query("バランは最後どうなったのか日本語で教えて", query_configs=query_configs2)

INFO:root:> Starting query: バランは最後どうなったのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30d0\\u30e9\\u30f3\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API r

DEBUG:root:>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 3), where each item in the list corresponds to a summary.
---------------------
(1)  十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配からモンスターが解き放たれ、世界に平和が訪れた。その後、赤子の時に漂着したデルムリン島で、鬼面道士のブラスに育てられた少年ダイは、友達のモンスターたちと共に平和に暮らしつつも、勇者になることを夢見ていた。そんな中、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受け、短期間のうちに秘められた力を開花させていくが、15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲する。ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける

(2)  レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち入ることが出来ずにいた。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、奇跡を起こす神々のアイテム「神の涙」であったゴメちゃんの最期の力によって、世界中の人々が協力し、爆発は阻止される。ダイは竜の騎士の戦闘形態である竜魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。制御を失い崩壊しながら浮上を続けるバーンパレスごと宇宙空間で繰り広げられる両者の最後の激闘は、ダイの勝利で終わる

(3) Ron Burk is certain that Dai is still alive somewhere, due to the magical gem on D

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 655
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3452 request_id=eb5c45ab2254b9a12971fdf3fd19253d response_code=200
DEBUG:root:
ANSWER: 6 and 10. 
These summaries provide the most relevant information to the question, as they both detail the events that happened to Balan in the final battle. Summary 6 explains how Balan sacrificed himself to protect the others, while Summary 10 explains how Balan's rival, Verza, was the true mastermind behind the assassin Kilburn, and how Balan was ultimately defeated by Dai.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10), where each item in the list corresponds to a summary.
---------------------
(1)   大魔王との対峙、交戦 軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も魔界の名工ロン・ベルクによる「ダイの剣」の作成や、アバンの戦友であった大魔道師マトリフと拳聖ブロキーナへの師事と

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=8032 request_id=1c5e366dda4f061d7b6ee21e1860614e response_code=200
DEBUG:root:

バランは、アバンがハドラーを倒した後、キルバーンの罠を躊躇なく身代わりになってダイを救おうとしたが、ダイは核晶を空高く運んでいく間に、ポップを地上に戻し、一人爆発の閃光の中に消えてしまったと思われます。数週間の捜索を経てもなお、ダイの姿は見つからなかったため、バランは最後には平和な死を遂げたと思われます。
DEBUG:root:> Refined response: 

バランは、アバンがハドラーを倒した後、キルバーンの罠を躊躇なく身代わりになってダイを救おうとしたが、ダイは核晶を空高く運んでいく間に、ポップを地上に戻し、一人爆発の閃光の中に消えてしまったと思われます。数週間の捜索を経てもなお、ダイの姿は見つからなかったため、バランは最後には平和な死を遂げたと思われます。
DEBUG:root:>[Level 1] Current answer response: 

バランは、アバンがハドラーを倒した後、キルバーンの罠を躊躇なく身代わりになってダイを救おうとしたが、ダイは核晶を空高く運んでいく間に、ポップを地上に戻し、一人爆発の閃光の中に消えてしまったと思われます。数週間の捜索を経てもなお、ダイの姿は見つからなかったため、バランは最後には平和な死を遂げたと思われます。 
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["The original question is as f

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["The original question is as follows: \\u30d0\\u30e9\\u30f3\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\nWe have provided an existing answer: \\n\\n\\u30d0\\u30e9\\u30f3\\u306f\\u3001\\u30a2\\u30d0\\u30f3\\u304c\\u30cf\\u30c9\\u30e9\\u30fc\\u3092\\u5012\\u3057\\u305f\\u5f8c\\u3001\\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306e\\u7f60\\u3092\\u8e8a\\u8e87\\u306a\\u304f\\u8eab\\u4ee3\\u308f\\u308a\\u306b\\u306a\\u3063\\u3066\\u30c0\\u30a4\\u3092\\u6551\\u304a\\u3046\\u3068\\u3057\\u305f\\u304c\\u3001\\u30c0\\u30a4\\u306f\\u6838\\u6676\\u3092\\u7a7a\\u9ad8\\u304f\\u904b\\u3093\\u3067\\u3044\\u304f\\u9593\\u306b\\u3001\\u30dd\\u30c3\\u30d7\\u3092\\u5730\\u4e0a\\u306b\\u623b\\u3057\\u3001\\u4e00\\u4eba\\u7206\\u767a\\u306e\\u9583\\u5149\\u306e\\u4e2d\\u306b\\u6d88\\u3048\\u30

Response(response='バランは、アバンがハドラーを倒した後、キルバーンの罠を躊躇なく身代わりになってダイを救おうとしたが、ダイは核晶を空高く運んでいく間に、ポップを地上に戻し、一人爆発の閃光の中に消えてしまったと思われます。数週間の捜索を経てもなお、ダイの姿は見つからなかったため、バランは最後には平和な死を遂げ、ダイを救うことができなかったと思われます。また、ダイが行方不明となった後、ダイの剣に付いている魔法の玉がまだ輝いていることから、ロン・バークはダイがまだ生きていると確信しています。そして、ダイがいつか帰ってくるまで、世界の平和を守ると誓ったポップは、ダイを再び探す旅に出ていきました。', source_nodes=[SourceNode(source_text='バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。\nダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった', doc_id='c11828fb-27fe-4607-bd05-61fe7c407177', extra_info=None, node_info={'start': 659, 'end': 1013}, similarity=None), SourceNode(source_text='ダイはハドラーを倒すが、その直後にキルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった。\n\n父・バランから受け継いだ竜の紋章の力でダイはバーンを追い詰めるが、バーンはミストバーンに

In [60]:
b_index.query("バランは最後どうなったのか日本語で教えて")

INFO:root:> Starting query: バランは最後どうなったのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.\\n---------------------\\n(1)  \\u5341\\u6570\\u5e74\\u524d\\u3001\\u9b54\\u738b\\u30cf\\u30c9\\u30e9\\u30fc\\u3092\\u5012\\u3057\\u305f\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u52dd\\u5229\\u306b\\u3088\\u308a\\u3001\\u9b54\\u738b\\u306e\\u652f\\u914d\\u304b\\u3089\\u30e2\\u30f3\\u30b9\\u30bf\\u30fc\\u304c\\u89e3\\u304d\\u653e\\u305f\\u308c\\u3001\\u4e16\\u754c\\u306b\\u5e73\\u548c\\u304c\\u8a2a\\u308c\\u305f\\u3002\\u305d\\u306e\\u5f8c\\u3001\\u8d64\\u5b50\\u306e\\u6642\\u306b\\u6f02\\u7740\\u3057\\u305f\\u30c7\\u30eb\\u30e0\\u30ea\\u30f3\\u5cf6\\u3067\\u3001\\u9b3c\\u9762\\u9053\\u58eb\\u306e\\u30d6\\u30e9\\u30b9\\u306b\\u80b2\\u3066\\u3089\\u308c\\u305f

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.\\n---------------------\\n(1)   \\u5927\\u9b54\\u738b\\u3068\\u306e\\u5bfe\\u5cd9\\u3001\\u4ea4\\u6226 \\u8ecd\\u56e3\\u95774\\u4eba\\u3092\\u6b20\\u304d\\u3001\\u9b54\\u738b\\u8ecd\\u306e\\u6226\\u529b\\u306f\\u5927\\u304d\\u304f\\u4f4e\\u4e0b\\u3057\\u305f\\u3053\\u3068\\u304b\\u3089\\u3001\\u30ec\\u30aa\\u30ca\\u306f\\u4e16\\u754c\\u5404\\u56fd\\u306e\\u738b\\u306b\\u5354\\u529b\\u3092\\u547c\\u3073\\u304b\\u3051\\u3001\\u9b54\\u738b\\u8ecd\\u3078\\u306e\\u6c7a\\u6226\\u306e\\u6e96\\u5099\\u3092\\u9032\\u3081\\u308b\\u3002\\u30c0\\u30a4\\u4e00\\u884c\\u3082\\u9b54\\u754c\\u306e\\u540d\\u5de5\\u30ed\\u30f3\\u30fb\\u30d9\\u30eb\\u30af\\u306b\\u3088\\u308b\\u300c\\u30c0\\u30a4\\u306e\\u5263\\u300d\\u306e\\u4f5c\\u

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 556
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2411 request_id=5b88c4feaea87873bcce878c9248adee response_code=200
DEBUG:root:
ANSWER: 6
This summary was selected because it provides information about the fate of Balan. It states that Balan sacrificed himself to protect the others from the explosion of the black core, and that his son, who had been secretly training, returned to help the group defeat the enemy.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.
---------------------
(1)   大魔王との対峙、交戦 軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も魔界の名工ロン・ベルクによる「ダイの剣」の作成や、アバンの戦友であった大魔道師マトリフと拳聖ブロキーナへの師事といった出来事を経て、新たな力を蓄えていく

(2) 一方、失態を重ねて奮起したハドラーは妖魔司教ザボエラの手で自らを超魔生物にさせ、その覚悟を認めたバーンから直属のハドラー親衛騎団も得る。  ダイ一行

Response(response='バランは、アバンがハドラーを倒した後、キルバーンの罠を躱したときに犠牲になりました。', source_nodes=[SourceNode(source_text='ダイはハドラーを倒すが、その直後にキルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった。\n\n父・バランから受け継いだ竜の紋章の力でダイはバーンを追い詰めるが、バーンはミストバーンに預けていた肉体を使って真の姿となる', doc_id='e10be0e1-ed27-4bf2-9e51-c19d9f9f90ef', extra_info=None, node_info=None, similarity=None)], extra_info=None)

In [61]:
b_index.query("バランはどうして死んだのか日本語で教えて")

INFO:root:> Starting query: バランはどうして死んだのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some choices are given below. It is provided in a numbered list (1 to 3),where each item in the list corresponds to a summary.\\n---------------------\\n(1)  \\u5341\\u6570\\u5e74\\u524d\\u3001\\u9b54\\u738b\\u30cf\\u30c9\\u30e9\\u30fc\\u3092\\u5012\\u3057\\u305f\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u52dd\\u5229\\u306b\\u3088\\u308a\\u3001\\u9b54\\u738b\\u306e\\u652f\\u914d\\u304b\\u3089\\u30e2\\u30f3\\u30b9\\u30bf\\u30fc\\u304c\\u89e3\\u304d\\u653e\\u305f\\u308c\\u3001\\u4e16\\u754c\\u306b\\u5e73\\u548c\\u304c\\u8a2a\\u308c\\u305f\\u3002\\u305d\\u306e\\u5f8c\\u3001\\u8d64\\u5b50\\u306e\\u6642\\u306b\\u6f02\\u7740\\u3057\\u305f\\u30c7\\u30eb\\u30e0\\u30ea\\u30f3\\u5cf6\\u3067\\u3001\\u9b3c\\u9762\\u9053\\u58eb\\u306e\\u30d6\\u30e9\\u30b9\\u306b\\u80b2\\u3066\\u3089\\u308c\\u305f

INFO:root:>[Level 0] Selected node: [2]/[2]
DEBUG:root:>[Level 0] Node [2] Summary text:  レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行は魔王軍の本拠地である鬼岩城の破壊に成功するものの、真の本拠地であるバーンパレスは魔宮の門によって硬く閉ざされ、立ち...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.\\n---------------------\\n(1)   \\u5927\\u9b54\\u738b\\u3068\\u306e\\u5bfe\\u5cd9\\u3001\\u4ea4\\u6226 \\u8ecd\\u56e3\\u95774\\u4eba\\u3092\\u6b20\\u304d\\u3001\\u9b54\\u738b\\u8ecd\\u306e\\u6226\\u529b\\u306f\\u5927\\u304d\\u304f\\u4f4e\\u4e0b\\u3057\\u305f\\u3053\\u3068\\u304b\\u3089\\u3001\\u30ec\\u30aa\\u30ca\\u306f\\u4e16\\u754c\\u5404\\u56fd\\u306e\\u738b\\u306b\\u5354\\u529b\\u3092\\u547c\\u3073\\u304b\\u3051\\u3001\\u9b54\\u738b\\u8ecd\\u3078\\u306e\\u6c7a\\u6226\\u306e\\u6e96\\u5099\\u3092\\u9032\\u3081\\u308b\\u3002\\

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 557
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2267 request_id=37a00ebc05d2c29e70dfde83e3754b01 response_code=200
DEBUG:root:
ANSWER: 6
This summary was selected because it explains how Balan sacrificed himself to protect Dai and his companions from the trap set by Kilburn. It also explains how Balan's last act was to use the power of the dragon crest he inherited from his father to confront and challenge Burn.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.
---------------------
(1)   大魔王との対峙、交戦 軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も魔界の名工ロン・ベルクによる「ダイの剣」の作成や、アバンの戦友であった大魔道師マトリフと拳聖ブロキーナへの師事といった出来事を経て、新たな力を蓄えていく

(2) 一方、失態を重ねて奮起したハドラーは妖魔司教ザボエラの手で自らを超魔生物にさせ、その覚悟を認めたバーンから直属のハドラー親衛騎団も得る。  ダイ一

Response(response='バランは、ハドラーを倒すために自分の肉体を賭けて戦った結果、死亡したと思われます。', source_nodes=[SourceNode(source_text='ダイはハドラーを倒すが、その直後にキルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった。\n\n父・バランから受け継いだ竜の紋章の力でダイはバーンを追い詰めるが、バーンはミストバーンに預けていた肉体を使って真の姿となる', doc_id='e10be0e1-ed27-4bf2-9e51-c19d9f9f90ef', extra_info=None, node_info=None, similarity=None)], extra_info=None)